# 🦜🔗 Langchain Demo

Welcome! This example will give you a first look at langchain. 

In [ ]:
%pip install -r requirements.txt

## Load Environment

Load ENV Variables from .env file

needed ENV vars:

```
# OpenAI Hosted Variante
OPENAI_API_KEY
OPENAI_ORGANIZATION (optional)

# Azure Hosted Variante
OPENAI_API_KEY=""
OPENAI_API_BASE="https://<your instance name>.openai.azure.com/"
OPENAI_API_TYPE="azure"
OPENAI_API_VERSION= "2023-05-15"
``` 

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Erster Test

ChatGPT oder auch jedes andere LLM benutzen ist relativ einfach mit Langchain

In diesen Test nutzen wir das "gpt-3.5-turbo" model - mögliche Large Language Modele von OpenAI sind:
- `gpt-3.5-turbo` // das model das am günstigsten und dadurch auch extrem verbreitet ist
- `gpt-3.5-turbo-16k` // das selbe model nur mit einem viel größeren "Gedächtnis"
- `gpt-4` // das neue und bessere GPT Model
- `gpt-4-32k` // wie das andere GPT-4 Model nur auch wieder mit einem 4 mal so großen "Gedächtnis"
- Demnächst wohl auch gpt-4-turbo

Diese Modelle gibt es auch alle noch einmal mit einem Zeitstempel, denn OpenAI bringt immer wieder neu trainierte Versionen der vorhandenen Modelle heraus.
Dies kann auch dazu führen, das Anfragen an das LLM nun komplett andere Antworten geben. 

Wenn man hier nicht aufpasst, kann es dazu führen, das Programme, die auf diesen Modellen basieren, nicht mehr funktionieren,
weil die passenden Marker im Text fehlen.

### Temperatur
Alle LLMs sind nicht deterministisch. Aber die Temperatur ist ein Parameter, mit der man die Variabilität von Antworten hoch und runter schrauben kann.
Wie bei normalen Atomen, wenn die Temperatur niedrig ist, ist die Bewegung niedrig, wenn man die Temperatur hochschraubt, wird viel gewackelt.
Hier ist der Parameter ein Fließkommawert zwischen 0 und 1.

### Links:
- https://www.langchain.com/
- https://python.langchain.com/docs/get_started/introduction
- https://platform.openai.com/docs/models/gpt-3-5
- https://platform.openai.com/docs/models/gpt-4
- https://platform.openai.com/docs/deprecations
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

In [ ]:
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
llm = AzureChatOpenAI(azure_deployment="GPT3")
# llm = ChatOpenAI(model_name="gpt-3.5-turbo")

print(llm.predict("Hi OpenAI! Kannst Du mir gerade mal einen plattdeutschen Trinkspruch kreiren?"))

## Tokens

Token sind die kleinste Einheit mit der LLMs arbeiten.
LLMs sind grob gesagt Wahrscheinlichkeitsmodele die das Token ausgeben das am wahrschneinlichsten zum vorherigen passt.
Tokens können Wörter, machmal sogar Wortgruppen oder auch nur einzelne oder mehrer Buchstaben.

So generieren LLMs die wahrschneinlichste fortführung der Eingabetoken
Its Statistik all the Way down

Links:
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb 

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

tokens = encoding.encode("AI ist eine tolle Sache.")

print(tokens)

decoded_tokens = [encoding.decode_single_token_bytes(token) for token in tokens]

print(decoded_tokens)

## Prompt Engineering und Templates in Langchain

Um die dinge von der AI zu bekommen die man wirklich will muss man oft sehr spezifisch Nachfragen und diese Nachfrage Templates
können wir selber erstellen oder im falle von Langchain auf Langchain Hub nachschauen und benutzen

Wenn man immer wieder und wieder den gleichen Text nur mit anderen Parametern haben will, kann man diese relativ einfach als PromptTemplate erstellen
und diese an das LLM übergeben.

### Links
- https://python.langchain.com/docs/get_started/quickstart#prompt-templates
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering
- https://learnprompting.org/docs/intro
- https://www.promptingguide.ai/
- https://smith.langchain.com/hub

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Du bist ein {beruf} aus Sankt Pauli."),
        ("human", "Erklär in 2 Sätzen in hamburger dialekt warum Deine Kunden aus {ort} die besten sind."),
    ]
)

print(prompt.format(beruf="Bäcker", ort="Pinneberg"))


### Langchain Hub Beispiel

Links:
- https://smith.langchain.com/hub/takatost/conversation-title-generator

In [ ]:
from langchain import hub
obj = hub.pull("takatost/conversation-title-generator")

print(obj.format(question="Was kann ich mittels AI in meiner Firma automatisieren?"))

## Die Chains die Langchain den Namen geben


### Links
- https://python.langchain.com/docs/modules/chains/how_to/async_chain

In [ ]:
import time
from langchain.chains import LLMChain
from langchain.schema import StrOutputParser

runnable = prompt | llm | StrOutputParser()

# einfacher aufruf
print(runnable.invoke({"beruf":"Bäcker", "ort":"Pinneberg"}))

# streaming aufruf
# for chunk in runnable.stream({"beruf":"Bäcker", "ort":"Pinneberg"}):
    # print(chunk, end="", flush=True)
    # time.sleep(0.1)

## Embeddings und Vectoren

Embeddings sind eine möglichkeit Texte in für CPU/GPUs besser verarbeitbare Vectoren umzuwandeln. Mit diesen Vectorn kann man dann so schöne dinge machen wie:
- Suche (wobei die Ergebnisse nach Relevanz für eine Abfragezeichenfolge geordnet werden)
- Clustering (wobei Textzeichenfolgen nach Ähnlichkeit gruppiert werden)
- Empfehlungen (wobei Elemente mit zugehörigen Textzeichenfolgen empfohlen werden)
- Anomalieerkennung (wobei Ausreißer mit geringem Zusammenhang identifiziert werden)
- Klassifizierung (wobei Textzeichenfolgen nach ihrer ähnlichsten Bezeichnung klassifiziert werden)

Die Standard Anwendung wäre hier das Umwandeln von Artikeln in einer Knowledge Datenbank in Vectoren, diese in einer Vectordatenbank zu hinterlegen und dann beim
Befragen eines Chatbots dann die anhand der Frage die "ähnlichsten" oder nächstgelegenden Dokumente zu finden und diese dann als Kontext dem Chatbot zum beantworten der Frage mitzugeben

Embeddings machen dies vorallem möglihch weil sie nicht auf genaue oder ungenaue Stringmatches arbeiten sondern wirklich eine Semantische nähe zueinander finden.
Ein Beispiel wäre das im Englischen Queen und King semantisch extrem nah sind aber in einer Stringmatch suche nicht zusammen zu finden sein sollten.
Das Berühmte Embedding Beispiel ist auch immer das "King - Man + Woman ~= Queen"

Es gibt sehr viele Embedding Modelle die für alle möglichen Fälle optimiert sind. Sie gibt es in Einsprachig und Mehrsprachig. Es gibt aber auch Multimodale Embeddings die z.B. für das Wort Schraube und das Bild einer Schraube sehr ähnliche Vektoren herausgeben. Es gibt auch welche die auf Sprache trainiert wurden.

Links:
- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
- https://python.langchain.com/docs/integrations/text_embedding/azureopenai
- https://app.twelvelabs.io/blog/multimodal-embeddings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(azure_deployment="Embeddings")
# embeddings = OpenAIEmbeddings()

text = "This is a test document."

query_result = embeddings.embed_query(text)

print("Dimensions: ",len(query_result))

# show the first 3 dimensions
print(query_result[:3])

### Einfaches Beispiel von Entfernungsberechnung mittels Vektoren
Kleinere Zahl = Näher an der Referenz

In [ ]:
from langchain.evaluation import EmbeddingDistance, load_evaluator
import pandas as pd

from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale

# Two lists of sentences
reference = 'Schraube'

sentences = [ 'Kreuzschlitz',
              'Torx',
              'Maggi',
              'Inbus',
              'Nagel',
              'Tempo',
            ]

evaluator = load_evaluator("embedding_distance", embeddings=embeddings)

distances = []
for sentence in sentences:
    distance = evaluator.evaluate_strings(prediction=sentence, reference=reference)
    distances.append(distance["score"])


df = pd.DataFrame({
    "Satz": sentences,
    "Entfernung": distances
})

renderers = {
 "Entfernung": BarRenderer(
        horizontal_alignment="center",
        bar_color=ColorScale(min=0, max=1, scheme="viridis"),
        bar_value=LinearScale(min=0, max=1),
    )
}

grid = DataGrid(df, base_column_size=250, renderers=renderers)
grid.transform(
    [
        {"type": "sort", "columnIndex": 2, "desc": False},
    ]
)

grid

### Debug Informationen gewünscht?

In [ ]:
from langchain.globals import set_debug, set_verbose
set_debug(False)

In [ ]:
## Und jetzt selber mal Ausprobieren

In [ ]:
print(chain.run({
    'beruf': "Programmierer",
    'ort': "Stuttgart"
    }))

Just go ahead and play a bit.